In [ ]:
import ase
import ase.visualize
import ase.io
from scipy import constants
import numpy as np
#%matplotlib
import matplotlib.pyplot as plt
#%matplotlib notebook
import copy
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
#this is just needed to change the font size in the figures
SMALL_SIZE = 8#12
MEDIUM_SIZE = 8#14
BIGGER_SIZE = 10#16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Read data 

### Define a few helpful functions

In [ ]:
def get_top_bottom(snapshot_in, surface_species):
    """Returns average position of the top and bottom atoms of a slab consisting of chemical species <surface_species> in the snapshot.
    Assumes a flat slab, with atoms in one layer spreading by no more than 1.4A and a vacuum spacing of at least the slab thikness.
    "top" is defined as the surface where the solvent lies in +z direction, bottom is where the solvent lies in -z direction.
    the position of 'bottom' will always be returned such as to lie in the upper half of the unit cell, that of 'top' in the lower half"""
    snapshot = copy.deepcopy(snapshot_in)
    shifted = False
    surface_mask = np.array([symbol == surface_species for symbol in snapshot.get_chemical_symbols()])
    top_atom_z = max(snapshot.get_positions(wrap = True)[surface_mask,2])
    bottom_atom_z = min(snapshot.get_positions(wrap = True)[surface_mask,2])
    #make sure the slab lies in one block in the unit cell to determine the true top and bottom
    if top_atom_z - bottom_atom_z > snapshot.cell[2,2]*0.5:
        snapshot.positions[:,2] += snapshot.cell[2,2]*0.5
        shifted = True
        top_atom_z = max(snapshot.get_positions(wrap = True)[surface_mask,2])
        bottom_atom_z = min(snapshot.get_positions(wrap = True)[surface_mask,2])
    top_index = np.where(snapshot.get_positions(wrap = True)[:,2] == top_atom_z)[0][0]
    bottom_index = np.where(snapshot.get_positions(wrap = True)[:,2] == bottom_atom_z)[0][0]
    mask_top = np.array([(atom.symbol == surface_species and abs(snapshot.get_distance(atom.index, top_index, mic=True, vector=True)[2])<1.4) for atom in snapshot])
    mask_bottom = np.array([(atom.symbol == surface_species and abs(snapshot.get_distance(atom.index, bottom_index, mic=True, vector=True)[2])<1.4) for atom in snapshot])
    z_top = np.mean(snapshot.get_positions(wrap=True)[mask_top, 2])
    z_bottom = np.mean(snapshot.get_positions(wrap=True)[mask_bottom, 2])
    if shifted:
        z_top -= snapshot.cell[2,2]*0.5
        z_bottom -= snapshot.cell[2,2]*0.5
    if z_bottom < snapshot.cell[2,2]*0.5: #I actually think this if statement is not necessary.
        z_bottom += snapshot.cell[2,2]

    return z_top, z_bottom

In [ ]:
def shift_top_layer_to_0(atoms, surface_species):
    """Shifts average position of top layer to 0"""
    zs = []
    for i,snapshot in enumerate(atoms):
        if i%100 ==0:
            print("      Analyzing snapshot {} of {}...".format(i, len(atoms)))
        z_top, z_bottom = get_top_bottom(snapshot, surface_species)
        zs.append(z_top)
    z_av = np.mean(np.array(zs))
    for snapshot in atoms:
        snapshot.positions -= z_av

### Reading data

In [ ]:
#give cell
x_max_bulk = 15.0
y_max_bulk = 15.0
z_max_bulk = 15.0
gamma_bulk = 90.000
#Read trajectory with 2Cs
print("Reading Trajectory...")
atoms_bulk = ase.io.read('C:/Users/doblhoffdierk/Downloads/trajectories_Au_100/trajectories_Au_100/trajectories_bulk.xyz',index=slice(int(equil/delta_t), None, int(delta/delta_t)))
print("Trajectory read")
print("Adding pbc...")
for i,snapshot in enumerate(atoms_bulk):
    #print(i,len(atoms))
    snapshot.cell = [[x_max_bulk,0,0], [y_max_bulk*np.cos(gamma_bulk*np.pi/180), y_max_bulk*np.sin(gamma_bulk*np.pi/180),0],[0,0,z_max_bulk]]
    snapshot.pbc=True

In [ ]:
#give cell
x_max = 14.425
y_max = 17.310
z_max = 48.62
gamma = 90.000
#read
delta_t = 5 #one snapshot every 5 timesteps in this trajectroy; don't change unless trajectroy is changed
equil = 5000 #equilibration time
delta = 10 #analyze every delta timesteps
print("Reading Trajectory...")
atoms = ase.io.read('C:/Users/doblhoffdierk/Downloads/trajectories_Au_100/trajectories_Au_100/trajectories_Cs_4.xyz',index=slice(int(equil/delta_t), None, int(delta/delta_t)))
print("Trajectory read")
print("Adding pbc...")
for i,snapshot in enumerate(atoms):
    #print(i,len(atoms))
    snapshot.cell = [[x_max,0,0], [y_max*np.cos(gamma*np.pi/180), y_max*np.sin(gamma*np.pi/180),0],[0,0,z_max]]
    snapshot.pbc=True
#shift top layer to 0
print("Added pbc")
print("Shifting slab to 0...")
shift_top_layer_to_0(atoms, 'Au')
print("Shifted slab to 0.")
#visualize 
#ase.visualize.view(atoms)

In [ ]:
ase.visualize.view(atoms)

In [ ]:
ase.io.write('snapshot.vasp',atoms[1479],  format='vasp')

In [ ]:
len(atoms)

In [ ]:
#Read trajectory with 2Cs
print("Reading Trajectory...")
atoms_2 = ase.io.read('C:/Users/doblhoffdierk/Downloads/trajectories_Au_100/trajectories_Au_100/trajectories_Cs_2.xyz',index=slice(int(equil/delta_t), None, int(delta/delta_t)))
print("Trajectory read")
print("Adding pbc...")
for i,snapshot in enumerate(atoms_2):
    #print(i,len(atoms))
    snapshot.cell = [[x_max,0,0], [y_max*np.cos(gamma*np.pi/180), y_max*np.sin(gamma*np.pi/180),0],[0,0,z_max]]
    snapshot.pbc=True
#shift top layer to 0
print("Added pbc")
print("Shifting slab to 0...")
shift_top_layer_to_0(atoms_2, 'Au')
print("Shifted slab to 0.")

# Analyze structure of solvation shell

#### Define Functions to analyse structure of the solvation shell of ions

In [ ]:
def map2sphere(positions):
    positions_on_sphere = []
    for x,y,z in positions:
        r = np.sqrt(x*x + y*y + z*z)
        theta = np.arccos(z/r)
        phi = np.arctan2(y,x)
        x_sphere = np.sin( theta ) * np.cos( phi )
        y_sphere = np.sin( theta ) * np.sin( phi )
        z_sphere = np.cos( theta )
        positions_on_sphere.append([x_sphere, y_sphere, z_sphere])
    return positions_on_sphere
        

In [ ]:
def get_angles_hull_simplices(atoms, ion, r_solv, z_start, z_end):
    """Constructs convex hull around all O atoms within r_solv from ions located in the region z_start, z_end.
    Computes angles of all simplices. Returns list of these angles and number of ions analyzed."""
    #loop through trajectory
    angles = []
    nions = 0
    for i,snapshot in enumerate(atoms):
        #if i%100 ==0:
            #print("      Analyzing snapshot {} of {}...".format(i, len(atoms)))
        #find ions in specified region
        ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>z_start and atom.position[2]<=z_end]
        o_indices = [atom.index for atom in snapshot if atom.symbol == 'O']
        #loop through ions to find the Os in their solvation shell and compute angles
        for ion_index in ion_indices:
            #find Os in solvation shell
            solvation_o_indices = [o_index for o_index in o_indices if snapshot.get_distances(o_index, ion_index, mic=True, vector=False) <= r_solv]
            if len(solvation_o_indices) >= 4: #need 4 points for convex hull
                nions += 1
                #find convex hull using minimum image convention and mapping all points onto sphere centered on ion with r=1.
                o_positions = snapshot.get_distances(ion_index, solvation_o_indices, mic=True, vector=True)
                o_positions_on_sphere = map2sphere(o_positions)
                o_positions_on_sphere += snapshot[ion_index].position
                o_positions += snapshot[ion_index].position
                hull = ConvexHull(o_positions)
                for s in hull.simplices:
                    s = np.append(s, s[0])  # Here we cycle back to the first coordinate
                    for i_line in range(len(s)-1):
                        i = s[i_line]
                        j = s[i_line + 1]
                        o_ion_o_angle = snapshot.get_angle(solvation_o_indices[i], ion_index, solvation_o_indices[j], mic=True)
                        angles.append(o_ion_o_angle)
    return np.array(angles), nions

In [ ]:
def get_O_ion_O_angles(atoms, ion, r_solv, z_start, z_end):
    """Identifies all O atoms within r_solv from ions located in the region z_start, z_end.
    Finds nearest O-O neighbors and computes their O-ion-O angle. Returns list of these angles and number of ions analyzed."""
    #loop through trajectory
    angles = []
    nions = 0
    for i,snapshot in enumerate(atoms):
        #if i%100 ==0:
            #print("      Analyzing snapshot {} of {}...".format(i, len(atoms)))
        #find ions in specified region
        ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>z_start and atom.position[2]<=z_end]
        o_indices = [atom.index for atom in snapshot if atom.symbol == 'O']
        #loop through ions to find the Os in their solvation shell and compute angles
        for ion_index in ion_indices:
            nions += 1
            #find Os in solvation shell
            solvation_o_indices = [o_index for o_index in o_indices if snapshot.get_distances(o_index, ion_index, mic=True, vector=False) <= r_solv]
            #find all nearest neighbors
            for i_o1 in range(len(solvation_o_indices)):
                for i_o2 in range(i_o1+1, len(solvation_o_indices)):
                    o_ion_o_angle = snapshot.get_angle(solvation_o_indices[i_o1], ion_index, solvation_o_indices[i_o2], mic=True)
                    angles.append(o_ion_o_angle)
    return np.array(angles), nions

In [ ]:
def get_nn_O_ion_O_angles(atoms, ion, r_solv, z_start, z_end):
    """Identifies all O atoms within r_solv from ions located in the region z_start, z_end.
    Finds nearest O-O neighbors and computes their O-ion-O angle. Returns list of these angles and number of ions analyzed."""
    #loop through trajectory
    angles = []
    nions = 0
    for i,snapshot in enumerate(atoms):
        #if i%100 ==0:
            #print("      Analyzing snapshot {} of {}...".format(i, len(atoms)))
        #find ions in specified region
        ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>z_start and atom.position[2]<=z_end]
        o_indices = [atom.index for atom in snapshot if atom.symbol == 'O']
        #loop through ions to find the Os in their solvation shell and compute angles
        for ion_index in ion_indices:
            nions += 1
            #find Os in solvation shell
            solvation_o_indices = [o_index for o_index in o_indices if snapshot.get_distances(o_index, ion_index, mic=True, vector=False) <= r_solv]
            #find all nearest neighbors
            for i_o1 in range(len(solvation_o_indices)):
                current_distance = 1000
                nn=-1
                for i_o2 in range(len(solvation_o_indices)):
                    #don't allow for angle to itself
                    if i_o1 == i_o2:
                        continue
                    distance = snapshot.get_distances(solvation_o_indices[i_o1], solvation_o_indices[i_o2], mic=True, vector=False)
                    if distance<current_distance:
                        current_distance = distance
                        nn = i_o2
                if nn>0: #NN found
                    o_ion_o_angle = snapshot.get_angle(solvation_o_indices[i_o1], ion_index, solvation_o_indices[nn], mic=True)
                    angles.append(o_ion_o_angle)
    return np.array(angles), nions

#### Analyze O-ion-O angle for all nearest neighbor Os in first solvation shell of ions

In [ ]:
#loop through z ranges for ion and compute nearest neighbor O - ion - O angles for O in first solvation shell of ion
z_bin_edges = [0.0, 4.0, 5.0, 6.2, 20.0]
hists = {}
nions = {}
data = {}
print("Getting histogramms...")
for i_bin in range(len(z_bin_edges)-1):
    print("   bin {} of {}...".format(i_bin, len(z_bin_edges)-1))
    z_start = z_bin_edges[i_bin]
    z_end = z_bin_edges[i_bin+1]
    label = "z{:3.1f}-{:3.1f}".format(z_start, z_end)
    #print("   {}".format(label))
    
    nn_O_ion_O_angles, nions_i = get_nn_O_ion_O_angles(atoms, 'Cs', 3.999, z_start, z_end)
    nions[label] = nions_i
    data[label] = nn_O_ion_O_angles
    hists[label], bin_edges = np.histogram(data[label], bins=72, range=[0,180], density=False)
    plt.plot(bin_edges[:-1]+0.5*(bin_edges[1]-bin_edges[0]), hists[label]/nions[label], label=label)
plt.legend()
print("Done.")

Conclusion: Hard to interpret, as much information is "thrown away" in this graph. Overall it seems that there is a slight shift to smaller angles, in particular for z=4-5A, when the coordination number is not reduced yet. The hull results belwo may be easier to interpret.

#### Analyze angle of all O-ion-O pairs for Os in first solvation shell of ions

In [ ]:
#loop through z ranges for ion and compute all O - ion - O angles for O in first solvation shell of ion
z_bin_edges = [0.0, 4.0, 5.0, 6.2, 20.0]
hists_all = {}
nions_all = {}
data_all = {}
print("Getting histogramms...")
for i_bin in range(len(z_bin_edges)-1):
    print("   bin {} of {}...".format(i_bin, len(z_bin_edges)-1))
    z_start = z_bin_edges[i_bin]
    z_end = z_bin_edges[i_bin+1]
    label = "z{:3.1f}-{:3.1f}".format(z_start, z_end)
    #print("   {}".format(label))
    
    nn_O_ion_O_angles, nions_i = get_O_ion_O_angles(atoms, 'Cs', 3.999, z_start, z_end)
    nions_all[label] = nions_i
    data_all[label] = nn_O_ion_O_angles
    hists_all[label], bin_edges_all = np.histogram(data_all[label], bins=72, range=[0,180], density=False)
    plt.plot(bin_edges_all[:-1]+0.5*(bin_edges_all[1]-bin_edges_all[0]), hists_all[label]/nions_all[label], label=label)
plt.legend()
print("Done.")

Conclusion: Hard to analyze as ALL angles are included (also the really large ones for Os in the solvation shell that are not neighbored. Hull results below may be easier to analyze.

#### Analyze O-ion-O angles along Os forming simplices of hull formed by Os in the solvation shell of ion

Note that this is the best "O-O neighbor" description for Os in the solvation shell that I could come up with

In [ ]:
#loop through z ranges for ion and compute O - ion - O angles for simplices of complex hull around O in first solvation shell of ion
z_bin_edges = [0.0, 4.0, 5.0, 6.2, 20.0]
hists_hull = {}
nions_hull = {}
data_hull = {}
colors = ['darkslategrey','teal','darkturquoise', 'turquoise']
fig, ax1 = plt.subplots(figsize=(3.3,2))
print("Getting histogramms...")
for i_bin in range(len(z_bin_edges)-1):
    print("   bin {} of {}...".format(i_bin, len(z_bin_edges)-1))
    z_start = z_bin_edges[i_bin]
    z_end = z_bin_edges[i_bin+1]
    label = "{:3.1f}<z<{:3.1f}".format(z_start, z_end)
    #print("   {}".format(label))
    
    nn_O_ion_O_angles, nions_i = get_angles_hull_simplices(atoms, 'Cs', 3.999, z_start, z_end)
    nions_hull[label] = nions_i
    data_hull[label] = nn_O_ion_O_angles
    hists_hull[label], bin_edges_hull = np.histogram(data_hull[label], bins=72, range=[0,180], density=False)
    plt.plot(bin_edges_hull[:-1]+0.5*(bin_edges_hull[1]-bin_edges_hull[0]), hists_hull[label]/nions_hull[label], label=label, color = colors[i_bin])
plt.legend()
plt.xlabel("O-ion-O angle [deg]")
plt.ylabel("probability [arb. units]")
plt.tight_layout()
fig.savefig('figures/O-ion-O_simplicesAngles.svg', format='svg', dpi=300)
print("Done.")

Conclusion: little difference between z>6.2 and 5<z<6.3 although there seems to be some changes for angles between 70 and 175 deg.
for 4<z<5 there is a clear shift to the small angles<50 deg.
for z<4 the small angles are slightly reduced (but there are also overall less Os in the solvation shell, so overall decrease of area under the curve! Additionally large angles (due to large opening angle) become clearly visible. Those seem to be mainly compensated by angles between 17 and 10 deg, rather than the really small angles.

#### Check influence of choice of r_cut

In [ ]:
#loop through z ranges for ion and compute O - ion - O angles for simplices of complex hull around O in first solvation shell of ion
z_bin_edges = [0.0, 4.0, 5.0, 6.2, 20.0]
hists_hull = {}
nions_hull = {}
data_hull = {}
print("Getting histogramms...")
for i_bin in range(len(z_bin_edges)-1):
    print("   bin {} of {}...".format(i_bin, len(z_bin_edges)-1))
    z_start = z_bin_edges[i_bin]
    z_end = z_bin_edges[i_bin+1]
    label = "z{:3.1f}-{:3.1f}".format(z_start, z_end)
    #print("   {}".format(label))
    
    nn_O_ion_O_angles, nions_i = get_angles_hull_simplices(atoms, 'Cs', 3.7, z_start, z_end)
    nions_hull[label] = nions_i
    data_hull[label] = nn_O_ion_O_angles
    hists_hull[label], bin_edges_hull = np.histogram(data_hull[label], bins=72, range=[0,180], density=False)
    plt.plot(bin_edges_hull[:-1]+0.5*(bin_edges_hull[1]-bin_edges_hull[0]), hists_hull[label]/nions_hull[label], label=label)
plt.legend()
print("Done.")

Conclusion: general results unchanged to the observations above.

#### Check implementation of hull

In [ ]:
def plot_angles_hull_simplices(atoms, ion, r_solv, z_start, z_end):
    """Constructs convex hull around all O atoms within r_solv from ions located in the region z_start, z_end.
    Computes angles of all simplices. Returns list of these angles and number of ions analyzed."""
    #loop through trajectory
    for i,snapshot in enumerate(atoms):
        #stop after 5 snapshots
        if i>1:
            break
        if i%100 ==0:
            print("      Analyzing snapshot {} of {}...".format(i, len(atoms)))
        #find ions in specified region
        ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>z_start and atom.position[2]<=z_end]
        o_indices = [atom.index for atom in snapshot if atom.symbol == 'O']
        #loop through ions to find the Os in their solvation shell and compute angles
        for ion_index in ion_indices:
            #find Os in solvation shell
            solvation_o_indices = [o_index for o_index in o_indices if snapshot.get_distances(o_index, ion_index, mic=True, vector=False) <= r_solv]
            if len(solvation_o_indices) >= 4: #need 4 points for convex hull
                #open figure
                fig = plt.figure()
                ax = fig.add_subplot(111, projection="3d")
                #plot ion
                print(snapshot[ion_index].position)
                ax.scatter([snapshot[ion_index].position[0]],[snapshot[ion_index].position[1]],[snapshot[ion_index].position[2]] , "ko") 
                #find convex hull using minimum image convention and mapping all points onto sphere with r=1.
                o_positions = snapshot.get_distances(ion_index, solvation_o_indices, mic=True, vector=True)
                o_positions_on_sphere = map2sphere(o_positions)
                o_positions_on_sphere += snapshot[ion_index].position
                o_positions += snapshot[ion_index].position
                hull = ConvexHull(o_positions_on_sphere)
                for s in hull.simplices:
                    s = np.append(s, s[0])  # Here we cycle back to the first coordinate
                    for i_line in range(len(s)-1):
                        i = s[i_line]
                        j = s[i_line + 1]
                        o_ion_o_angle = snapshot.get_angle(solvation_o_indices[i], ion_index, solvation_o_indices[j], mic=True)
                        #plot convex hull color coded for angles
                        if o_ion_o_angle<55:
                            color = 'r-'
                        else:
                            color = 'b-'
                        #print(np.array([o_positions[i], o_positions[j]]))
                        pts = np.array([o_positions[i], o_positions[j]])
                        ax.plot(pts[:,0], pts[:,1], pts[:,2], color)
                plt.show()
                #plot geometry around ion
                h_indices = [atom.index for atom in snapshot if atom.symbol == 'H']
                solvation_h_indices = []
                for o_index in solvation_o_indices:
                    solvation_h_indices += [h_index for h_index in h_indices if snapshot.get_distances(o_index, h_index, mic=True, vector=False) <= 1.2]
                print(solvation_h_indices)
                ase.visualize.view(snapshot[solvation_o_indices+solvation_h_indices+[ion_index]])
            
                        

In [ ]:
#loop through z ranges for ion and compute O - ion - O angles for simplices of complex hull around O in first solvation shell of ion
z_bin_edges = [0.0, 4.0]#, 5.0, 6.2, 20.0]
for i_bin in range(len(z_bin_edges)-1):
    print("   bin {} of {}...".format(i_bin, len(z_bin_edges)-1))
    z_start = z_bin_edges[i_bin]
    z_end = z_bin_edges[i_bin+1]
    label = "z{:3.1f}-{:3.1f}".format(z_start, z_end)
    print("   {}".format(label))
    
    plot_angles_hull_simplices(atoms, 'Cs', 3.999, z_start, z_end)

Conclusion: The hull is identified similarly to what I would have done by hand. The plots above are difficult to see in 3D but by turning on %matplotlib notebook above, the structures can be turned and become clearer. This, however, tends to crash my notebook...

# Analyze surface-near water

### Define functions

In [ ]:
def get_waters_ltZA(atoms, z):
    nsnapshots = 0
    nOs = 0
    for snapshot in atoms:
        o_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2] < z]
        nsnapshots += 1
        nOs += len(o_indices)
    return float(nOs)/float(nsnapshots)

In [ ]:
def get_water_angle(atoms, maxz, nz=20, every = 1):
    n_waters = np.zeros(nz)
    angles = np.zeros(nz)
    zs = np.arange(0,maxz,maxz/nz)
    zs = zs + 0.5 * (zs[1]-zs[0])
    max_bond = 1.2
    for i,snapshot in enumerate(atoms):
        if i%every == 0:
            #print(i, len(atoms))
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2] < maxz]
            h_indices = [atom.index for atom in snapshot if atom.symbol == 'H' and atom.position[2] < maxz + max_bond]
            for o_index in o_indices:
                # find location of o to correctly place in array
                iz = int(snapshot[o_index].position[2]/maxz*nz)
                water_h_indices = [h_index for h_index in h_indices if snapshot.get_distances(o_index, h_index, mic=True, vector=False) <= max_bond]
                intercept = snapshot.get_distances(o_index, water_h_indices[0], mic=True, vector=True) + snapshot.get_distances(o_index, water_h_indices[1], mic=True, vector=True)
                angle = np.arccos(np.dot(intercept, np.array([0,0,1])) / np.linalg.norm(intercept)) * 180 / np.pi
                angles[iz] += angle
                n_waters[iz] += 1
    return zs, angles/n_waters
                

In [ ]:
def get_water_OH_angle(atoms, maxz, nz=20, every = 1, direction = 'down'):
    n_waters = np.zeros(nz)
    angles = np.zeros(nz)
    zs = np.arange(0,maxz,maxz/nz)
    zs = zs + 0.5 * (zs[1]-zs[0])
    max_bond = 1.2
    for i,snapshot in enumerate(atoms):
        if i%every == 0:
            #print(i, len(atoms))
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2] < maxz]
            h_indices = [atom.index for atom in snapshot if atom.symbol == 'H' and atom.position[2] < maxz + max_bond]
            for o_index in o_indices:
                # find location of o to correctly place in array
                iz = int(snapshot[o_index].position[2]/maxz*nz)
                water_h_indices = [h_index for h_index in h_indices if snapshot.get_distances(o_index, h_index, mic=True, vector=False) <= max_bond]
                v1 = snapshot.get_distances(o_index, water_h_indices[0], mic=True, vector=True)
                ang1 = np.arccos(np.dot(v1, np.array([0,0,1])) / np.linalg.norm(v1)) * 180 / np.pi
                v2 = snapshot.get_distances(o_index, water_h_indices[1], mic=True, vector=True)
                ang2 = np.arccos(np.dot(v2, np.array([0,0,1])) / np.linalg.norm(v2)) * 180 / np.pi
                if direction == 'down':
                    #get most downward OH bond
                    if ang2 > ang1:
                        ang1 = ang2
                elif direction == 'up':
                    #get most up OH bond
                    if ang2 < ang1:
                        ang1 = ang2
                else:
                    print("Unknown direction. Using random angle")
                angles[iz] += ang1
                n_waters[iz] += 1
    return zs, angles/n_waters

In [ ]:
def get_water_density(atoms, maxz, nz=20, every = 1):
    n_snapshots = 0
    density = np.zeros(nz)
    zs = np.arange(0,maxz,maxz/nz)
    zs = zs + 0.5 * (zs[1]-zs[0])
    for i, snapshot in enumerate(atoms):
        if i%every == 0:
            n_snapshots += 1
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2] < maxz]
            for o_index in o_indices:
                # find location of o to correctly place in array
                iz = int(snapshot[o_index].position[2]/maxz*nz)
                density[iz] += 1
    density = density / n_snapshots
    #unit conversion
    A = np.linalg.norm(np.cross(atoms[0].get_cell()[0,0:2], atoms[0].get_cell()[1,0:2]))
    density *= (2*ase.Atom('H').mass + ase.Atom('O').mass) / A / (zs[1]-zs[0])
    density *= constants.physical_constants["atomic mass constant"][0] * 1E27 #kg/dm^3 = g/cm^3
    return zs, density

In [ ]:
def get_probability_species(atoms, species, maxz, nz=20, every = 1):
    density = np.zeros(nz)
    zs = np.arange(0,maxz,maxz/nz)
    zs = zs + 0.5 * (zs[1]-zs[0])
    for i, snapshot in enumerate(atoms):
        if i%every == 0:
            indices = [atom.index for atom in snapshot if atom.symbol == species and atom.position[2] < maxz]
            for index in indices:
                # find location of o to correctly place in array
                iz = int(snapshot[index].position[2]/maxz*nz)
                density[iz] += 1
    #normalize
    density = density / sum(density) * nz / maxz
    return zs, density

### Analyze Data

#### Water denisty profile

In [ ]:
#density
print("Analyzing angle 4Cs")
zs, density = get_water_density(atoms, 10, nz=100)
print("Analyzing angle 2Cs")
zs_2, density_2 = get_water_density(atoms_2, 10, nz=100)
plt.plot(zs, density, label = "4 Cs+")
plt.plot(zs_2, density_2, label = "2 Cs+")
plt.legend()
plt.xlabel("distance [A]")
plt.ylabel("density [kg/l]")

#### Counting water close to interface per cell

In [ ]:
#count waters <5A
cut = 5
waters_4Cs = get_waters_ltXA(atoms, cut)
waters_2Cs = get_waters_ltXA(atoms_2, cut)
print("4Cs: average water number <{} in box = {}".format(cut, waters_4Cs))
print("2Cs: average water number <{} in box = {}".format(cut, waters_2Cs))
print("Percentage change = {}%".format((waters_4Cs-waters_2Cs)/waters_2Cs*100))

In [ ]:
#count waters <4.5A
cut = 4.5
waters_4Cs = get_waters_ltXA(atoms, cut)
waters_2Cs = get_waters_ltXA(atoms_2, cut)
print("4Cs: average water number <{} in box = {}".format(cut, waters_4Cs))
print("2Cs: average water number <{} in box = {}".format(cut, waters_2Cs))
print("Percentage change = {}%".format((waters_4Cs-waters_2Cs)/waters_2Cs*100))

In [ ]:
#count waters <4A
cut = 4
waters_4Cs = get_waters_ltXA(atoms, cut)
waters_2Cs = get_waters_ltXA(atoms_2, cut)
print("4Cs: average water number <{} in box = {}".format(cut, waters_4Cs))
print("2Cs: average water number <{} in box = {}".format(cut, waters_2Cs))
print("Percentage change = {}%".format((waters_4Cs-waters_2Cs)/waters_2Cs*100))

Conclusion: Number of water molecules in first hydration layer of the metal is slightly decreased (~10% depeinding on the exact cutoff distance) when there are more ions. The largest changes come from distances between 4 and 4.5A, in the water density minimum (which is lower with more ions present - see also plot above)

#### ion density profile

In [ ]:
zs_prob, ion_prob = get_probability_species(atoms, 'Cs', 10, nz=100)
zs_2_prob, ion_prob_2 = get_probability_species(atoms_2, 'Cs', 10, nz=100)
plt.plot(zs_prob, ion_prob, label = "4 Cs+")
plt.plot(zs_2_prob, ion_prob_2, label = "2 Cs+")
plt.legend()
plt.xlabel("distance [A]")
plt.ylabel("probability")

In [ ]:
fig, ax1 = plt.subplots(figsize=(2.8,2))
ax2 = ax1.twinx()
c1 = 'teal'
c2 = 'firebrick'
ax1.plot([0,0.1], [0,0.0], label = r'4 Cs$^+$', color='k', ls='-', lw=1.5)
ax1.plot([0,0.1], [0,0.0], label = r"2 Cs$^+$", color='k', ls=':', lw=1.5)
ax1.legend()
ax1.plot(zs, density, label = "4 Cs+", color=c1, ls='-', lw=1.5)
ax1.plot(zs_2, density_2, label = "2 Cs+", color=c1, ls=':', lw=1.5)
ax2.plot(zs_prob, ion_prob*4, label = "4 Cs+", color = c2 , ls='-', lw=1.5)
ax2.fill_between(zs_prob,0, ion_prob*4, color = c2, alpha = 0.15)
ax2.plot(zs_2_prob, ion_prob_2*2, label = "2 Cs+", color = c2, ls=':', lw=1.5)
ax2.fill_between(zs_2_prob, 0,ion_prob_2*2,color = c2, alpha = 0.15)
ax1.set_xlabel("distance $z$ [A]")
ax1.set_ylabel("water density [kg/l]", color=c1)
ax2.set_ylabel("ion number density", color=c2)
ax1.spines["left"].set_edgecolor(c1)
ax2.spines["left"].set_edgecolor(c1)
ax1.spines["right"].set_edgecolor(c2)
ax2.spines["right"].set_edgecolor(c2)
ax2.set_ylim(0,4)
ax1.set_ylim(0,3.2)
ax1.tick_params(axis='y', colors=c1)
ax2.tick_params(axis='y', colors=c2)
plt.tight_layout()
fig.savefig('figures/density.svg', format='svg', dpi=300)

#### Analyzing water angle for water <5A

intercept vs. z (0deg is up, 180 deg is down)

In [ ]:
#angle
zmax = 5
print("Analyzing angle 4Cs")
zs, angles = get_water_angle(atoms, zmax, nz=20, every = 10)
print("Analyzing angle 2Cs")
zs_2, angles_2 = get_water_angle(atoms_2, zmax, nz=20, every = 10)
plt.plot(zs, angles, label = "4 Cs")
plt.plot(zs_2, angles_2, label = "2 Cs")
plt.legend()
plt.xlabel("distance [A]")
plt.ylabel("intercept angle [deg]")
plt.xlim(0,zmax)

average most downward OH angle (0deg is up, 180 deg is down)

In [ ]:
#OH-down angle
print("Analyzing angle 4Cs")
zs, angles = get_water_OH_angle(atoms, 5, nz=20, every = 10, direction = 'down')
print("Analyzing angle 2Cs")
zs_2, angles_2 = get_water_OH_angle(atoms_2, 5, nz=20, every = 10, direction = 'down')
plt.plot(zs, angles, label = "4 Cs")
plt.plot(zs_2, angles_2, label = "2 Cs")
plt.legend()
plt.xlabel("distance [A]")
plt.ylabel("OH angle [deg]")
plt.xlim(0,zmax)

average most upward OH angle (0deg is up, 180 deg is down)

In [ ]:
#OH-up angle
print("Analyzing angle 4Cs")
zs, angles = get_water_OH_angle(atoms, 5, nz=20, every = 10, direction = 'up')
print("Analyzing angle 2Cs")
zs_2, angles_2 = get_water_OH_angle(atoms_2, 5, nz=20, every = 10, direction = 'up')
plt.plot(zs, angles, label = "4 Cs")
plt.plot(zs_2, angles_2, label = "2 Cs")
plt.legend()
plt.xlabel("distance [A]")
plt.ylabel("OH angle [deg]")
plt.xlim(0,zmax)

Conclusion: Up to 4A form the interface, one OH bond points mainly downwards, whle the other points mainly horizontal (4Cs) or slightly up on average (2 Cs). For 2Cs, the very closest H2O molecules point one bond nearly horizontal and one slightly up on average. This is likey due to chemisorbed water types.

# Analyzing H-bonding

### Define a few functions

In [ ]:
#From https://journals.aps.org/prb/abstract/10.1103/PhysRevB.110.014105:
#"An H-bond is defined if the distance between the donor oxygen (Od) and acceptor oxygen (Oa) 
# is less than 3.5 ˚ A and the O-O-H angle, ∠OaOdHd, is less than 30° [S19]"

def get_Hbond_angle_dist(atoms, maxz, nz=20, nangle = 20,every = 1, angle = True):
    """Returns 2D  arrays x, y and an array with the average number of H-donors. 
    x describes the z-position of the donor (acceptors if acceptor_perspective == True) 
    if angle == True: y describes the hydrogen bond angle to the accptor 
    if angle == False: y describes the z-position of the acceptor 
    """
    #set counter for snapshots (to normalize the data in the end)
    n_snapshots = 0
    #definition of H-bond
    max_OO_dist = 3.5
    max_OH_bond = 1.2
    max_angle = 30
    #set up arrays
    zs = np.arange(0,maxz,maxz/nz)
    zs = zs + 0.5 * (zs[1]-zs[0])
    if angle == True:
        angles = np.arange(0,180,180/nangle)
        angles = angles + 0.5 * (angles[1]-angles[0])
        y,x = np.meshgrid(angles, zs)
    else: 
        z2s = np.arange(0,maxz + max_OO_dist, maxz / nz)
        z2s = z2s + 0.5 * (zs[1]-zs[0])
        y,x = np.meshgrid(z2s, zs)
        nangle = len(z2s) #needed to correctly get the size of n_donors
    n_donors = np.zeros((nz, nangle))
        
    for i,snapshot in enumerate(atoms):
        if i%every == 0:
            n_snapshots += 1
            #find possible donors and candidats for H that bond to them
            o_donor_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2] < maxz]
            h_indices = [atom.index for atom in snapshot if atom.symbol == 'H' and atom.position[2] < maxz + max_OH_bond]
            #find candidates for acceptors
            o_acceptor_candidate_indices = np.array([atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2] < maxz + max_OO_dist])
            for o_donor_index in o_donor_indices:
                #find the Hs bonding to this O
                water_h_indices = [h_index for h_index in h_indices if snapshot.get_distances(o_donor_index, h_index, mic=True, vector=False) <= max_OH_bond]
                #find acceptors matching the distance requirement
                oo_distances = snapshot.get_distances(o_donor_index, o_acceptor_candidate_indices, mic=True, vector=False)
                o_acceptor_indices = o_acceptor_candidate_indices[oo_distances < max_OO_dist]
                #cycle through all possible o_acceptors and check angle criterion
                for o_acceptor_index in o_acceptor_indices:
                    #skip self
                    if o_acceptor_index == o_donor_index:
                        continue
                    for i_h in range(2):
                        ang = snapshot.get_angle(o_acceptor_index, o_donor_index, water_h_indices[i_h], mic=True)
                        if ang < max_angle: #identified as H-bond
                            # find location of o to correctly place in array     
                            iz = int(snapshot[o_donor_index].position[2]/maxz*nz)
                            #find location in y array (angle or z-position of binding partner)
                            if angle == True:
                                #compute angle of H-bond relative to z and get index of angle
                                vec = snapshot.get_distance(o_donor_index, o_acceptor_index, mic=True, vector=True)
                                ang = np.arccos(np.dot(vec, np.array([0,0,1])) / np.linalg.norm(vec)) * 180 / np.pi
                                iangle = int(ang/180*nangle)
                            else: #angle == False --> return z-position of binding partner
                                # find location of o to correctly place in array       
                                iangle = int(snapshot[o_acceptor_index].position[2]/maxz*nz) #this works as deltaz is the same for z1 and z2.
                            n_donors[iz,iangle] += 1
                            break #second H cannot have H-bond to same o_accoptor
                
    return x,y,n_donors/n_snapshots

## Analyze

### Analyze average number of H-donors with a specific O_donor to O_acceptor direction vs. z

(0deg means the O-O connection lies horizontal, positive angles are up, negative angles are down)

#### 4Cs in unit cell

In [ ]:
#H-donor distribution
print("Analyzing angle 4Cs")
zmax = 8
n_bins = 40
x,y,n_donors = get_Hbond_angle_dist(atoms, zmax, nz=n_bins, nangle = n_bins, every = 10)
fig, ax = plt.subplots()
c = ax.pcolormesh(x, -(y-90), n_donors, cmap='Blues', vmin=0, vmax=n_donors.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()
#print("Analyzing angle 2Cs")
#x2,y2,n_donors3, angles_2 = get_Hbond_angle_dist(atoms_2, 5, every = 10)


#### 2 Cs in unit cell

In [ ]:
#H-donor distribution
print("Analyzing angle 2Cs")
x_2,y_2,n_donors_2 = get_Hbond_angle_dist(atoms_2, zmax, nz=n_bins, nangle = n_bins, every = 10)
fig, ax = plt.subplots()
c = ax.pcolormesh(x_2, -(y_2-90), n_donors_2, cmap='Blues', vmin=0, vmax=n_donors.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()
#print("Analyzing angle 2Cs")
#x2,y2,n_donors3, angles_2 = get_Hbond_angle_dist(atoms_2, 5, every = 10)

#### difference

In [ ]:
#compare
from matplotlib.cm import get_cmap
fig, ax = plt.subplots()
vmax = np.max(np.abs(n_donors-n_donors_2))
RdBu_r = cmap_reversed = get_cmap('RdBu_r')
c = ax.pcolormesh(x, -(y-90), n_donors-n_donors_2, cmap=RdBu_r, vmin=-vmax, vmax=vmax, shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()

#### Analyze difference in hydration, separating first hydration shell (z<4.5) from rest

In [ ]:
print("Difference between 4 Cs in unit cell and 2 Cs in unit cell:")
print("Total difference in number of H-donors = {}".format(sum(sum(n_donors-n_donors_2))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Difference in number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[:cut,:]-n_donors_2[:cut,:]))))
print("Difference in number of H-donors up to z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[cut:,:]-n_donors_2[cut:,:]))))

In [ ]:
print("Absolute numbers 2 Cs in unit cell:")
print("Total number of H-donors = {}".format(sum(sum(n_donors_2))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_2[:cut,:]))))
print("Total number of H-donors up to z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_2[cut:,:]))))

In [ ]:
print("Absolute numbers 4 Cs in unit cell:")
print("Total number of H-donors = {}".format(sum(sum(n_donors))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[:cut,:]))))
print("Total number of H-donors up to z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[cut:,:]))))

In [ ]:
print("Relative change in number of H-donors for z<= {} = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]), sum(sum(n_donors[:cut,:]-n_donors_2[:cut,:]))/sum(sum(n_donors_2[:cut,:]))*100))

Conclusion: A strong change in number of hydrogen bonds formed within first layer (see numbers right above and difference plot a bit further above) for the 4 Cs unit cells compared to the 2 Cs unit cells

### Analyze position of H-acceptors for H-donors at different positions

#### 4Cs cell

In [ ]:
#H-donor distribution
print("Analyzing 4Cs")
zmax = 8
n_bins = 40
x_pos,y_pos,n_donors_pos = get_Hbond_angle_dist(atoms, zmax, nz=n_bins, every = 10, angle = False)
fig, ax = plt.subplots()
c = ax.pcolormesh(x_pos, y_pos, n_donors_pos, cmap='Blues', vmin=0, vmax=n_donors_pos.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, 0, zmax + 3.5])
fig.colorbar(c, ax=ax)
plt.axis('scaled')
plt.grid()
plt.show()

#### 2Cs cell

In [ ]:
#H-donor distribution
print("Analyzing 2Cs")
zmax = 8
n_bins = 40
x_pos,y_pos,n_donors_2_pos = get_Hbond_angle_dist(atoms_2, zmax, nz=n_bins, every = 10, angle = False)
fig, ax = plt.subplots()
c = ax.pcolormesh(x_pos, y_pos, n_donors_2_pos, cmap='Blues', vmin=0, vmax=n_donors_pos.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, 0, zmax + 3.5])
fig.colorbar(c, ax=ax)
plt.axis('scaled')
plt.grid()
plt.show()

#### difference

In [ ]:
#compare
from matplotlib.cm import get_cmap
fig, ax = plt.subplots()
vmax = np.max(np.abs(n_donors_pos-n_donors_2_pos))
RdBu_r = cmap_reversed = get_cmap('RdBu_r')
c = ax.pcolormesh(x_pos, y_pos, n_donors_pos-n_donors_2_pos, cmap=RdBu_r, vmin=-vmax, vmax=vmax, shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, 0, zmax + 3.5])
fig.colorbar(c, ax=ax)
plt.axis('scaled')
plt.grid()
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, (ax,ax2) = plt.subplots(1,2, figsize=(2.8,2.5), sharey=True,)
c = ax.pcolormesh(x_pos, y_pos, n_donors_pos, cmap='Blues', vmin=0, vmax=n_donors_pos.max(),shading= 'auto')
c2 = ax2.pcolormesh(x_pos, y_pos, n_donors_pos-n_donors_2_pos, cmap=RdBu_r, vmin=-vmax, vmax=vmax, shading= 'auto')
# set the limits of the plot to the limits of the data
#ax.axis([0, zmax, 0, zmax + 3.5])
#fig.colorbar(c, ax=[ax], location = 'top')
#fig.colorbar(c, ax=[ax],orientation="horizontal")
divider = make_axes_locatable(ax)
cax = divider.append_axes('top', size='5%', pad=0.1)
cb =fig.colorbar(c, cax=cax, orientation='horizontal')
cb.ax.xaxis.set_ticks_position("top")
divider = make_axes_locatable(ax2)
cax2 = divider.append_axes('top', size='5%', pad=0.1)
cb2= fig.colorbar(c2, cax=cax2, orientation='horizontal')
cb2.ax.xaxis.set_ticks_position("top")

#fig.colorbar(c2, ax=ax2)
ax.axis('scaled')
ax2.axis('scaled')
ax.axis([2, zmax, 2, zmax +1])
ax2.axis([2, zmax, 2, zmax +1])
#ax.grid()
ax.set_xticks([2,4,6,8])
ax2.set_xticks([2,4,6,8])
ax.set_yticks([2,4,6,8])
ax2.set_yticks([2,4,6,8])
ax.set_xlabel(r'$z$ donor [$\AA$]', labelpad=0)
ax2.set_xlabel(r'$z$ donor [$\AA$]', labelpad=0)
ax.set_ylabel(r'$z$ acceptor [$\AA$]')
plt.tight_layout()
plt.show()
fig.savefig('figures/hbonding.svg', format='svg', dpi=300)

In [ ]:
print("Difference between 4 Cs in unit cell and 2 Cs in unit cell:")
print("Total difference in number of H-donors = {}".format(sum(sum(n_donors_pos-n_donors_2_pos))))
#donor above z=4.5
cut = np.where(x_pos[:,0]>=4.5)[0][0]
#acceptor below z=4.5
cut2 = np.where(y_pos[0,:]>=4.5)[0][0]
print("Difference in number of H-donors for z_donor >= {} and z_accept <= {} = {}".format(x_pos[cut,0]-0.5*(x_pos[1,0]-x_pos[0,0]), y_pos[0,cut2]-0.5*(y_pos[0,1]-y_pos[0,0]),  sum(sum(n_donors_pos[cut:,:cut2]-n_donors_2_pos[cut:,:cut2]))))

In [ ]:
print("4 Cs:")
print("Total number of H-donors = {}".format(sum(sum(n_donors_pos))))
print("Number of H-donors for z_donor >= {} and z_accept <= {} = {}".format(x_pos[cut,0]-0.5*(x_pos[1,0]-x_pos[0,0]), y_pos[0,cut2]-0.5*(y_pos[0,1]-y_pos[0,0]),  sum(sum(n_donors_pos[cut:,:cut2]))))

In [ ]:
print("2 Cs:")
print("Total number of H-donors = {}".format(sum(sum(n_donors_2_pos))))
print("Number of H-donors for z_donor >= {} and z_accept <= {} = {}".format(x_pos[cut,0]-0.5*(x_pos[1,0]-x_pos[0,0]), y_pos[0,cut2]-0.5*(y_pos[0,1]-y_pos[0,0]),  sum(sum(n_donors_2_pos[cut:,:cut2]))))

In [ ]:
print("Relative change in number of H-donors for z_donor >= {} and z_accept <= {} = {}".format(x_pos[cut,0]-0.5*(x_pos[1,0]-x_pos[0,0]), y_pos[0,cut2]-0.5*(y_pos[0,1]-y_pos[0,0]),  sum(sum(n_donors_pos[cut:,:cut2]-n_donors_2_pos[cut:,:cut2]))/sum(sum(n_donors_2_pos[cut:,:cut2]))*100))

In [ ]:
print("Difference between 4 Cs in unit cell and 2 Cs in unit cell:")
cut = np.where(x_pos[:,0]>=5)[0][0]
cut_e = np.where(x_pos[:,0]>=7.5)[0][0]
cut2 = np.where(y_pos[0,:]>=5)[0][0]
cut2_e = np.where(y_pos[0,:]>=7.5)[0][0]
print("Difference in number of H-donors for {} <= z_donor <= {} and {} <= z_accept <= {} = {}".format(x_pos[cut,0]-0.5*(x_pos[1,0]-x_pos[0,0]),x_pos[cut_e,0]-0.5*(x_pos[1,0]-x_pos[0,0]),  y_pos[0,cut2]-0.5*(y_pos[0,1]-y_pos[0,0]), y_pos[0,cut2_e]-0.5*(y_pos[0,1]-y_pos[0,0]), sum(sum(n_donors_pos[cut:cut_e,cut2:cut2_e]-n_donors_2_pos[cut:cut_e,cut2:cut2_e]))))

Conclusion: Reduced h-bonding from second layer to first layer, enhanced h-bonding within 2nd layer

### Do the same, but selecting snapshots with more or less ions close to the interface

#### Define a few functions

In [ ]:
def get_number_of_snapshots_n_ions_lt_z(atoms, ion, zmax):
    n_snapshots = np.zeros(10)
    for j, snapshot in enumerate(atoms):
        ion_indices = [atom.index for atom in snapshot if atom.symbol == ion if atom.position[2]<zmax]
        n_snapshots[len(ion_indices)] += 1
    return n_snapshots

#### The following is usefull to check how many snapshots have a certain number of ions in the regeon <z specified in last variable

In [ ]:
print(get_number_of_snapshots_n_ions_lt_z(atoms, 'Cs', 4.5))

In [ ]:
print(get_number_of_snapshots_n_ions_lt_z(atoms, 'Cs', 4.7))

In [ ]:
print(get_number_of_snapshots_n_ions_lt_z(atoms, 'Cs', 4.0))

#### extract snapshots with minimal 3 ions for z<4.0 and for maximal 2 ions for z<4.5 to distinguish regions with many and few ions close to interface

Note: A separation leading to (in my eyes) clearer results is made belwo (see next section).

In [ ]:
atoms_min3ions_lt_40 = []
atoms_max2ions_lt_45 = []
for snapshot in atoms:
    ion_indices = [atom.index for atom in snapshot if atom.symbol == 'Cs' if atom.position[2]<4.0]
    if len(ion_indices) >= 3:
        atoms_min3ions_lt_40.append(snapshot)
    ion_indices = [atom.index for atom in snapshot if atom.symbol == 'Cs' if atom.position[2]<4.5]
    if len(ion_indices) <=2:
        atoms_max2ions_lt_45.append(snapshot)

In [ ]:
#H-donor distribution
print("Analyzing angle 4Cs min 3 ions <= 4.0")
x,y,n_donors_min3ions_lt_40 = get_Hbond_angle_dist(atoms_min3ions_lt_40, zmax, nz=n_bins, nangle = n_bins, every = 10)
fig, ax = plt.subplots()
c = ax.pcolormesh(x, -(y-90), n_donors_min3ions_lt_40, cmap='Blues', vmin=0, vmax=n_donors.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()
#print("Analyzing angle 2Cs")
#x2,y2,n_donors3, angles_2 = get_Hbond_angle_dist(atoms_2, 5, every = 10)

In [ ]:
#H-donor distribution
print("Analyzing angle 4Cs max 2 ions <= 4.5")
x,y,n_donors_max2ions_lt_45 = get_Hbond_angle_dist(atoms_max2ions_lt_45, zmax, nz=n_bins, nangle = n_bins, every = 10)
fig, ax = plt.subplots()
c = ax.pcolormesh(x, -(y-90), n_donors_max2ions_lt_45, cmap='Blues', vmin=0, vmax=n_donors.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()
#print("Analyzing angle 2Cs")
#x2,y2,n_donors3, angles_2 = get_Hbond_angle_dist(atoms_2, 5, every = 10)

In [ ]:
print("2 Cs ions in unit cell")
print("Total number of H-donors = {}".format(sum(sum(n_donors_2_pos))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_2_pos[:cut,:]))))
print("Total number of H-donors up to z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_2_pos[cut:,:]))))

In [ ]:
#4Cs
print("4 Cs ions in unit cell")
print("Total number of H-donors = {}".format(sum(sum(n_donors))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[:cut,:]))))
print("Total number of H-donors for z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[cut:,:]))))

In [ ]:
#4Cs with min 3 Cs lt 4.0
print("4 Cs ions in unit cell with min 3 ions at z<4.0")
print("Total number of H-donors = {}".format(sum(sum(n_donors_min3ions_lt_40))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_min3ions_lt_40[:cut,:]))))
print("Total number of H-donors for z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_min3ions_lt_40[cut:,:]))))

In [ ]:
#4Cs with max 3 Cs ls 4.5
print("4 Cs ions in unit cell with max 2 ions at z<4.5 (2 ions for z>4.5)")
print("Total number of H-donors = {}".format(sum(sum(n_donors_max2ions_lt_45))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_max2ions_lt_45[:cut,:]))))
print("Total number of H-donors for z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_max2ions_lt_45[cut:,:]))))

Conclusion: Slight increase in hydrogen bonding (towards the 2 Cs situation) when there are less ions close to the interface. The separation made in the following section gives clearer restuls though in my eyes. Refer to those.

#### extract snapshots with minimal 4 ions for z<4.7 and for maximal 2 ions for z<4.7 to distinguish regions with many and few ions close to interface

In [ ]:
atoms_min4ions_lt_47 = []
atoms_max2ions_lt_47 = []
for snapshot in atoms:
    ion_indices = [atom.index for atom in snapshot if atom.symbol == 'Cs' if atom.position[2]<4.7]
    if len(ion_indices) >= 4:
        atoms_min4ions_lt_47.append(snapshot)
    elif len(ion_indices) <= 2:
        atoms_max2ions_lt_47.append(snapshot)

In [ ]:
#H-donor distribution
print("Analyzing angle 4Cs max 2 ions <= 4.7")
x,y,n_donors_max2ions_lt_47 = get_Hbond_angle_dist(atoms_max2ions_lt_47, zmax, nz=n_bins, nangle = n_bins, every = 10)
fig, ax = plt.subplots()
c = ax.pcolormesh(x, -(y-90), n_donors_max2ions_lt_47, cmap='Blues', vmin=0, vmax=n_donors.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()
#print("Analyzing angle 2Cs")
#x2,y2,n_donors3, angles_2 = get_Hbond_angle_dist(atoms_2, 5, every = 10)

In [ ]:
#H-donor distribution
print("Analyzing angle 4Cs min 4 ions <= 4.7")
x,y,n_donors_min4ions_lt_47 = get_Hbond_angle_dist(atoms_min4ions_lt_47, zmax, nz=n_bins, nangle = n_bins, every = 10)
fig, ax = plt.subplots()
c = ax.pcolormesh(x, -(y-90), n_donors_min4ions_lt_47, cmap='Blues', vmin=0, vmax=n_donors.max(),shading= 'auto')
# set the limits of the plot to the limits of the data
ax.axis([0, zmax, -90, 90])
fig.colorbar(c, ax=ax)
plt.grid()
plt.show()
#print("Analyzing angle 2Cs")
#x2,y2,n_donors3, angles_2 = get_Hbond_angle_dist(atoms_2, 5, every = 10)

In [ ]:
print("2 Cs in unit cell")
print("Total number of H-donors = {}".format(sum(sum(n_donors_2_pos))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_2_pos[:cut,:]))))
print("Total number of H-donors up to z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_2_pos[cut:,:]))))

In [ ]:
#4Cs
print("4 Cs in unit cell")
print("Total number of H-donors = {}".format(sum(sum(n_donors))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[:cut,:]))))
print("Total number of H-donors for z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors[cut:,:]))))

In [ ]:
print("Relative change: 4 Cs in unit cell vs. 2 Cs in unit cell")
print("Relative change in number of H-donors for z<= {} 4Cs vs 2Cs= {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]), sum(sum(n_donors[:cut,:]-n_donors_2[:cut,:]))/sum(sum(n_donors_2[:cut,:]))*100))
print("Relative change in number of H-donors for z<= {} 4Cs(4 ions <4.7) vs 2Cs= {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]), sum(sum(n_donors_min4ions_lt_47[:cut,:]-n_donors_2[:cut,:]))/sum(sum(n_donors_2[:cut,:]))*100))
print("Relative change in number of H-donors for z<= {} 4Cs(2 ions <4.7) vs 2Cs= {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]), sum(sum(n_donors_max2ions_lt_47[:cut,:]-n_donors_2[:cut,:]))/sum(sum(n_donors_2[:cut,:]))*100))

In [ ]:
#4Cs with min 4 Cs lt 4.7
print("4 Cs in unit cell with 4 at z<4.7A")
print("Total number of H-donors = {}".format(sum(sum(n_donors_min4ions_lt_47))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_min4ions_lt_47[:cut,:]))))
print("Total number of H-donors for z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_min4ions_lt_47[cut:,:]))))

In [ ]:
#4Cs with max 2 Cs lt 4.7
print("4 Cs in unit cell with max 2 at z<4.7")
print("Total number of H-donors = {}".format(sum(sum(n_donors_max2ions_lt_47))))
cut = np.where(x[:,0]>=4.5)[0][0]
print("Total number of H-donors up to z<={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_max2ions_lt_47[:cut,:]))))
print("Total number of H-donors for z>={} (ind = {}) = {}".format(x[cut,0]-0.5*(x[1,0]-x[0,0]),cut, sum(sum(n_donors_max2ions_lt_47[cut:,:]))))

Conclusion: The ion position seems to have an impact on the reduction of H-bonding in the first layer (not only the surface charge). For snapshots with less ions close to the surface, but the same overall surface charge, the reduction in H-bonding is reduced from 1/3 to less than 1/4, i.e. a nearly 30% decrease in decrease.

# Cs solvation

### Define a few functions

In [ ]:
def get_ion_location(atoms, ion, every = 1):
    n_ions = len([atom.index for atom in atoms[0] if atom.symbol == ion])
    z_ions = np.zeros((n_ions, len(atoms)))
    for j, snapshot in enumerate(atoms):
        if j%every == 0:
            #find all ions
            ion_indices = [atom.index for atom in snapshot if atom.symbol == ion]
            for i, ion_index in enumerate(ion_indices):
                z_ions[i,j]=snapshot[ion_index].position[2]
    return z_ions

In [ ]:
def get_coordination_number(atoms, ion, r_solv, every = 1):
    n_ions = len([atom.index for atom in atoms[0] if atom.symbol == ion])
    coord_number = np.zeros((n_ions, len(atoms)))
    for j, snapshot in enumerate(atoms):
        if j%every == 0:
            #find all ions
            ion_indices = [atom.index for atom in snapshot if atom.symbol == ion]
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O']
            #loop through ions to find the Os in their solvation shell and compute angles
            for i, ion_index in enumerate(ion_indices):
                #find Os in solvation shell
                solvation_o_indices = [o_index for o_index in o_indices if snapshot.get_distances(o_index, ion_index, mic=True, vector=False) <= r_solv]
                coord_number[i,j] = len(solvation_o_indices)
    #print(coord_number)
    return coord_number

In [ ]:
def get_average_coordination_number(atoms, ion, r_solv, every = 1, bins = [0,1000], i_ion = 'all'):
    n_ions = np.zeros(len(bins)-1)
    coord_number = np.zeros(len(bins)-1)
    #loop over snapshots
    for j, snapshot in enumerate(atoms):
        if j%every == 0:
            #loop over separate ranges
            for i in range(len(bins)-1):
                zmin = bins[i]
                zmax = bins[i+1]
                #find all ions
                ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>=zmin and atom.position[2]<zmax]
                o_indices = [atom.index for atom in snapshot if atom.symbol == 'O'and atom.position[2]>=zmin-r_solv and atom.position[2]<zmax+r_solv]
                #loop through ions to find the Os in their solvation shell
                for k, ion_index in enumerate(ion_indices):
                    if not i_ion == 'all':
                        if not i_ion == k+1:
                            continue
                    solvation_o_indices = [o_index for o_index in o_indices if snapshot.get_distances(o_index, ion_index, mic=True, vector=False) <= r_solv]
                    coord_number[i] += len(solvation_o_indices)
                    n_ions[i] += 1
    return np.array(coord_number)/np.array(n_ions), n_ions

In [ ]:
def get_rdf(atoms, ion, every = 1, rmax = 8, nbins = 20, zmin=0, zmax=1000):
    """Note that, to plot the cdf, one needs to add 0.5*bin_width to rs, 
    to get a reasonable plot (because this is integrated over the entire bin width, 
    but rs reports bin center"""
    rdf = np.zeros(nbins)
    rs = np.arange(0, rmax+rmax/nbins, rmax/nbins) 
    volumes = np.zeros(nbins)
    for i in range(len(rs)-1):
        volumes[i] = 4/3*np.pi*(rs[i+1]**3-rs[i]**3)
    nrdfs = 0
    for j, snapshot in enumerate(atoms):
        if j%every == 0:
            ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>=zmin and atom.position[2]<zmax]
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O']
            for ion_index in ion_indices:
                nrdfs += 1
                distances = snapshot.get_distances(ion_index, o_indices, mic=True)
                for distance in distances:
                    ir = int(distance * nbins / rmax)
                    if ir < nbins:
                        #print(distance-rs[ir]- 0.5*rmax/nbins)
                        rdf[ir] += 1
    #correctly normalize rdf
    rdf /= nrdfs
    cdf = np.cumsum(rdf)
    rdf /= volumes
    rdf *= (2*ase.Atom('H').mass + ase.Atom('O').mass)
    rdf *= constants.physical_constants["atomic mass constant"][0] * 1E27 #kg/dm^3 = g/cm^3
    #shift r to center of the bins and delete right corner
    rs = rs[:-1] + 0.5*rmax/nbins
    return rs, rdf, cdf

In [ ]:
def get_opening_angle(atoms, ion, r_cut, nbins, zmin = 0, zmax = 1000, every = 1):
    thetas = np.arange(0,180, 180/nbins) + 0.5*180/nbins
    n_thetas = np.zeros(len(thetas))
    for j, snapshot in enumerate(atoms):
        if j%every == 0:
            ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>=zmin and atom.position[2]<zmax]
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2]>=zmin-r_cut and atom.position[2]<=zmax+r_cut]
            for ion_index in ion_indices:
                distances = snapshot.get_distances(ion_index, o_indices, mic=True, vector = True)
                for distance in distances:
                    if np.linalg.norm(distance) > r_cut: #not in first solvation shell
                        continue
                    angle = np.arccos(np.dot(distance, np.array([0,0,-1])) / np.linalg.norm(distance)) * 180 / np.pi
                    iangle = int(angle * nbins / 180)
                    n_thetas[iangle] += 1
    #normalize
    n_thetas /= np.sum(n_thetas) 
    n_thetas *= nbins / 180
    return thetas, n_thetas
    

In [ ]:
def get_min_opening_angle(atoms, ion, r_cut, nbins, zmin = 0, zmax = 1000, every = 1):
    thetas = np.arange(0,180, 180/nbins) + 0.5*180/nbins
    n_thetas = np.zeros(len(thetas))
    for j, snapshot in enumerate(atoms):
        if j%every == 0:
            ion_indices = [atom.index for atom in snapshot if atom.symbol == ion and atom.position[2]>=zmin and atom.position[2]<zmax]
            o_indices = [atom.index for atom in snapshot if atom.symbol == 'O' and atom.position[2]>=zmin-r_cut and atom.position[2]<=zmax+r_cut]
            for ion_index in ion_indices:
                distances = snapshot.get_distances(ion_index, o_indices, mic=True, vector = True)
                angle_min = 180
                for distance in distances:
                    if np.linalg.norm(distance) > r_cut: #not in first solvation shell
                        continue
                    angle = np.arccos(np.dot(distance, np.array([0,0,-1])) / np.linalg.norm(distance)) * 180 / np.pi
                    if angle < angle_min:
                        angle_min = angle
                iangle = int(angle_min * nbins / 180)
                n_thetas[iangle] += 1
    #normalize
    n_thetas /= np.sum(n_thetas) 
    n_thetas *= nbins / 180
    return thetas, n_thetas

## Analyze

### Coordination number

#### check whether ions tend to remain in one region only

In [ ]:
every = 1
zions = get_ion_location(atoms, 'Cs', every = every)
fig=plt.figure(figsize = (6.6,4))
plt.plot(np.arange(len(zions[0,:]))*every*delta, zions[0,:], label= 'ion 1')
plt.plot(np.arange(len(zions[0,:]))*every*delta, zions[1,:], label= 'ion 2')
plt.plot(np.arange(len(zions[0,:]))*every*delta, zions[2,:], label= 'ion 3')
plt.plot(np.arange(len(zions[0,:]))*every*delta, zions[3,:], label= 'ion 4')
plt.plot([0, len(zions[0,:])*every*delta], [4,4], 'k')
plt.plot([0, len(zions[0,:])*every*delta], [5,5], 'k')
plt.plot([0, len(zions[0,:])*every*delta], [6.2,6.2], 'k')
plt.text(4000, 3, "region 1")
plt.text(4000, 4.5, "region 2")
plt.text(4000, 5.6, "region 3")
plt.text(4000, 7, "region 4")
plt.xlabel('time [fs]')
plt.ylabel(r'position of ion [$\AA$]')
plt.legend()
fig.savefig('figures/IonPosOverTime.svg', format='svg', dpi=300)

Conclusion: Switches occur mainly between region 1 and 2, but ion 1 remains in region 1 nearly throughout.

#### view coordination number over time

In [ ]:
#plot coordination number over time
every = 1
fig, (ax,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(6.6,4), sharex=True,)
coord_number = get_coordination_number(atoms, 'Cs', 3.999, every= every)
plt.figure(figsize = (8,4))
ax.plot(np.arange(len(coord_number[0,:]))*every*delta, coord_number[0,:], label = 'ion 1')
ax2.plot(np.arange(len(coord_number[0,:]))*every*delta, coord_number[1,:], label = 'ion 2, shifte by 5')
ax3.plot(np.arange(len(coord_number[0,:]))*every*delta, coord_number[2,:], label = 'ion 3, shifte by 10')
ax4.plot(np.arange(len(coord_number[0,:]))*every*delta, coord_number[3,:], label = 'ion 4, shifte by 15')
ax4.set_xlabel('time [fs]')
ax.set_ylabel('CN ion 1')
ax2.set_ylabel('CN ion 2')
ax3.set_ylabel('CN ion 3')
ax4.set_ylabel('CN ion 4')
fig.savefig('figures/CNoverTime.svg', format='svg', dpi=300)

Conclusion: CN changes strongly with "oscillation periods" of roughly 2ps (gauged by eye) --> probably need at least 8ps of data for the CN to be more or less reliable

#### Check difference in coordination number of ion 1 and 2 (which both stay close to the interface)

In [ ]:
r_cut = 3.999
av_coor_number_Cs1, tmp = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = [0,1000], i_ion = 1)
av_coor_number_Cs2, tmp = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = [0,1000], i_ion = 2)
av_coor_number_Cs3, tmp = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = [0,1000], i_ion = 3)
av_coor_number_Cs4, tmp = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = [0,1000], i_ion = 4)
print(av_coor_number_Cs1, av_coor_number_Cs2, av_coor_number_Cs3, av_coor_number_Cs4)

Conclusion: CN for ions 1 and 2 behave similarly. That is good, as ion 1 (with the higher CN) stays in region 1 throughout, but ion 2 oscillates between the regions. This gives some confidence in statistics, at least for long enough times in which ions are present in a region

#### Compute coordination number using a r_cut of 3.999

Also check how many ps are present in which region

In [ ]:
r_cut = 3.999
bins = [0,4,5,6.2,100]
every = 1
av_coor_number_binned, n_ions = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = bins)
av_coor_number_2_binned, n_ions_2 = get_average_coordination_number(atoms_2, 'Cs', r_cut, every = every, bins = bins)
av_coor_number_bulk, n_ions_bulk = get_average_coordination_number(atoms_bulk, 'Cs', r_cut, every = every, bins = [0,1000])
#av_coor_number_bulk = get_average_coordination_number(atoms_bulk, 'Cs', 3.999, every = every)
print("coordination number")
print("range        \t 4Cs \t 2Cs \t bulk")
for i in range(len(bins)-1):
    if i<len(bins)-2:
        print("{:5.1f}-{:5.1f}A:\t {:4.2f}\t{:4.2f}".format(bins[i], bins[i+1], av_coor_number_binned[i], av_coor_number_2_binned[i]))
    else:
        print("{:5.1f}-{:5.1f}A:\t {:4.2f}\t{:4.2f}\t{:4.2f}".format(bins[i], bins[i+1], av_coor_number_binned[i], av_coor_number_2_binned[i], av_coor_number_bulk[0]))
print("fs in which an ion is present in this region")
print("range        \t  4Cs  \t 2Cs  \t bulk")
for i in range(len(bins)-1):
    if i<len(bins)-2:
        print("{:5.1f}-{:5.1f}A:\t {:6.0f}\t{:6.0f}".format(bins[i], bins[i+1], n_ions[i]*every*delta, n_ions_2[i]*every*delta))
    else:
        print("{:5.1f}-{:5.1f}A:\t {:6.0f}\t{:6.0f}\t{:6.0f}".format(bins[i], bins[i+1], n_ions[i]*every*delta, n_ions_2[i]*every*delta, n_ions_bulk[0]*every*delta))
plt.plot(av_coor_number_binned, label = "4 Cs")
plt.plot(av_coor_number_2_binned, label = "2 Cs")
plt.plot([av_coor_number_bulk,av_coor_number_bulk,av_coor_number_bulk,av_coor_number_bulk], label = "bulk")
plt.legend()
plt.grid()

#### Check whether a larger r_cut changes things significantly

In [ ]:
r_cut = 4.2
bins = [0,4,5,6.2,100]
every = 10
av_coor_number_binned, n_ions = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = bins)
av_coor_number_2_binned, n_ions_2 = get_average_coordination_number(atoms_2, 'Cs', r_cut, every = every, bins = bins)
av_coor_number_bulk, n_ions_bulk = get_average_coordination_number(atoms_bulk, 'Cs', r_cut, every = every, bins = [0,1000])
#av_coor_number_bulk = get_average_coordination_number(atoms_bulk, 'Cs', 3.999, every = every)
print("coordination number")
print("range        \t 4Cs \t 2Cs \t bulk")
for i in range(len(bins)-1):
    if i<len(bins)-2:
        print("{:5.1f}-{:5.1f}A:\t {:3.1f}\t{:3.1f}".format(bins[i], bins[i+1], av_coor_number_binned[i], av_coor_number_2_binned[i]))
    else:
        print("{:5.1f}-{:5.1f}A:\t {:3.1f}\t{:3.1f}\t{:3.1f}".format(bins[i], bins[i+1], av_coor_number_binned[i], av_coor_number_2_binned[i], av_coor_number_bulk[0]))
print("fs in which an ion is present in this region")
print("range        \t  4Cs  \t 2Cs  \t bulk")
for i in range(len(bins)-1):
    if i<len(bins)-2:
        print("{:5.1f}-{:5.1f}A:\t {:6.0f}\t{:6.0f}".format(bins[i], bins[i+1], n_ions[i]*every*delta, n_ions_2[i]*every*delta))
    else:
        print("{:5.1f}-{:5.1f}A:\t {:6.0f}\t{:6.0f}\t{:6.0f}".format(bins[i], bins[i+1], n_ions[i]*every*delta, n_ions_2[i]*every*delta, n_ions_bulk[0]*every*delta))
plt.plot(av_coor_number_2_binned, label = "2 Cs")
plt.plot([av_coor_number_bulk,av_coor_number_bulk,av_coor_number_bulk,av_coor_number_bulk], label = "bulk")
plt.legend()
plt.grid()

Conclusion: Larger cutoff makes all changes a bit bigger, but qualitative picture remains the same

#### check whether smaller r_cut changes a lot

In [ ]:
r_cut = 3.8
bins = [0,4,5,6.2,100]
every = 10
av_coor_number_binned, n_ions = get_average_coordination_number(atoms, 'Cs', r_cut, every = every, bins = bins)
av_coor_number_2_binned, n_ions_2 = get_average_coordination_number(atoms_2, 'Cs', r_cut, every = every, bins = bins)
av_coor_number_bulk, n_ions_bulk = get_average_coordination_number(atoms_bulk, 'Cs', r_cut, every = every, bins = [0,1000])
#av_coor_number_bulk = get_average_coordination_number(atoms_bulk, 'Cs', 3.999, every = every)
print("coordination number")
print("range        \t 4Cs \t 2Cs \t bulk")
for i in range(len(bins)-1):
    if i<len(bins)-2:
        print("{:5.1f}-{:5.1f}A:\t {:4.2f}\t{:4.2f}".format(bins[i], bins[i+1], av_coor_number_binned[i], av_coor_number_2_binned[i]))
    else:
        print("{:5.1f}-{:5.1f}A:\t {:4.2f}\t{:4.2f}\t{:4.2f}".format(bins[i], bins[i+1], av_coor_number_binned[i], av_coor_number_2_binned[i], av_coor_number_bulk[0]))
print("fs in which an ion is present in this region")
print("range        \t  4Cs  \t 2Cs  \t bulk")
for i in range(len(bins)-1):
    if i<len(bins)-2:
        print("{:5.1f}-{:5.1f}A:\t {:6.0f}\t{:6.0f}".format(bins[i], bins[i+1], n_ions[i]*every*delta, n_ions_2[i]*every*delta))
    else:
        print("{:5.1f}-{:5.1f}A:\t {:6.0f}\t{:6.0f}\t{:6.0f}".format(bins[i], bins[i+1], n_ions[i]*every*delta, n_ions_2[i]*every*delta, n_ions_bulk[0]*every*delta))
plt.plot(av_coor_number_2_binned, label = "2 Cs")
plt.plot(av_coor_number_binned, label = "4 Cs")
plt.plot([av_coor_number_bulk,av_coor_number_bulk,av_coor_number_bulk,av_coor_number_bulk], label = "bulk")
plt.legend()
plt.grid()

### RDF and CDF

compute RDF and CDF

In [ ]:
rs_bulk, rdf_bulk, cdf_bulk = get_rdf(atoms_bulk, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=0, zmax=1000)
rs, rdf, cdf = get_rdf(atoms, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=0, zmax=1000)
rs_r1, rdf_r1, cdf_r1 = get_rdf(atoms, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=0, zmax=4)
rs_r2, rdf_r2, cdf_r2 = get_rdf(atoms, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=4, zmax=5)
rs_r3, rdf_r3, cdf_r3 = get_rdf(atoms, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=5, zmax=6.2)
rs_r4, rdf_r4, cdf_r4 = get_rdf(atoms, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=6.2, zmax=1000)
rs_2, rdf_2, cdf_2 = get_rdf(atoms_2, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=0, zmax=1000)
rs_2_r2, rdf_2_r2, cdf_2_r2 = get_rdf(atoms_2, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=4, zmax=5)
rs_2_r3, rdf_2_r3, cdf_2_r3 = get_rdf(atoms_2, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=5, zmax=6.2)
rs_2_r4, rdf_2_r4, cdf_2_r4 = get_rdf(atoms_2, 'Cs', every = 1, rmax = 8, nbins = 100, zmin=6.2, zmax=1000)

In [ ]:
#### Compare bulk, 2Cs and 4Cs

In [ ]:
plt.plot(rs_bulk, rdf_bulk, label = 'bulk')
plt.plot(rs_2, rdf_2, label = '2Cs')
plt.plot(rs, rdf, label = '4Cs')
plt.legend()
plt.grid()
#plt.xlim(3,5)
#plt.ylim(0,13);

zoom in 

In [ ]:
plt.plot(rs_bulk, rdf_bulk, label = 'bulk')
plt.plot(rs_2, rdf_2, label = '2Cs')
plt.plot(rs, rdf, label = '4Cs')
plt.legend()
plt.grid()
plt.xlim(0,8)
#plt.ylim(0,13);

compare 'bulk-type' only

In [ ]:
plt.plot(rs_bulk, rdf_bulk, label = 'bulk')
plt.plot(rs__2r4, rdf_2_r4, label = '2Cs region 4')
plt.plot(rs_r4, rdf_r4, label = '4Cs region 4')
plt.legend()
plt.grid()
plt.xlim(3.5,4.5)
plt.ylim(0.8, 1.6)

Conclusion: minimum in RDF is not clearly visible. It occurs around 4 for 4Cs, but rather at 4.2 for the bulk case and 2Cs. However, noise could easily influence the exact location of the minimum (in fact it is not 100% clear whether the ninimum is a result so noise only). The choice of the exact cutoff value is therefore somewhat arbitrary. Essentially, it could be set anywhere between 3.8 (where the rdf has fallen to 1-1.35, i.e. to something close to 1, where the "bulk" contribution will be large already) to 4.2. This is why above the analysis was repeated for r_cut = 3.8 and r_cut = 4.2. It is clear that, the larger the solvation radius is chosen, the larger the effect will be of going close to the surface. Except for the exact value of "desolvation", nothing much changes.

In [ ]:
#### Compare all bulk regions

In [ ]:
plt.plot(rs_bulk+0.5*(rs_bulk[1]-rs_bulk[0]), cdf_bulk,  '-x', label = 'bulk')
plt.plot(rs_2_r4+0.5*(rs_2_r4[1]-rs_2_r4[0]), cdf_2_r4, '-x',label = '2Cs region 4')
plt.plot(rs_r4+0.5*(rs_r4[1]-rs_r4[0]), cdf_r4, '-x',label = '4Cs region 4')
plt.legend()
plt.grid()
plt.xlim(2.5,4.2)
plt.ylim(0,11);

Conclusion: Clearly, the 4Cs case behaves somehwat differently from the other 2 cases. The reason may be the comparably bad statistics for this datapoint (only 5000 fs - see above)

#### Compare the cdf for the different regions for the 4 ion case

In [ ]:
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
c4 = 'paleturquoise'
c3 = 'darkturquoise'
c2 = 'teal'
c1 = 'darkslategrey'
fig, ax1 = plt.subplots(figsize=(2.8,2))
ax1.add_patch(Rectangle((0, 0), 3.999, 13, facecolor = 'gainsboro',fill=True, alpha=0.5))
plt.plot(rs_bulk, cdf_bulk, label = 'bulk', lw = 1, color='k')
plt.plot(rs_r1, cdf_r1, label = r'$0<z<4$', color = c1, lw = 1.5)
plt.plot(rs_r2, cdf_r2, label = r'4<z<5', color = c2, lw = 1.5)
plt.plot(rs_r3, cdf_r3, label = r'$5<z<6.2$', color = c3, lw = 1.5)
plt.plot(rs_r4, cdf_r4, label = r'$z>6.2$',  color = c4, lw = 1.5)
#plt.legend(loc=2)
#plt.grid()
plt.xlabel(r'Cs - O distance [$\AA$]')
plt.ylabel('cumm. distribution funct.')
ax1.set_ylim(0,10)
ax1.set_xlim(2.75,4.5)
axins = zoomed_inset_axes(ax1, 1.4,loc=4, bbox_to_anchor=(0.58,0.12,0.5,0.5),bbox_transform=ax1.transAxes)
axins.add_patch(Rectangle((0, 0), 3.999, 13, facecolor = 'gainsboro',fill=True, alpha=0.5))
axins.plot(rs_bulk, cdf_bulk, label = 'bulk', lw = 1, color='k')
axins.plot(rs_r1, cdf_r1, label = r'$0<z<4$', color = c1, lw = 1.5)
axins.plot(rs_r2, cdf_r2, label = r'4<z<5', color = c2, lw = 1.5)
axins.plot(rs_r3, cdf_r3, label = r'$5<z<6.2$', color = c3, lw = 1.5)
axins.plot(rs_r4, cdf_r4, label = r'$z>6.2$',  color = c4, lw = 1.5)
mark_inset(ax1, axins, loc1=3, loc2=1, fc="none", ec="0.5")
axins.grid()
axins.set_xlim(3.75, 4.25)
axins.set_ylim(6.5, 9.5)
#ax1.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
plt.tight_layout()
fig.savefig('figures/cdf.svg', format='svg', dpi=300)

In [ ]:
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
c4 = 'paleturquoise'
c3 = 'darkturquoise'
c2 = 'teal'
c1 = 'darkslategrey'
fig, ax1 = plt.subplots(figsize=(2.8,2))
ax1.add_patch(Rectangle((0, 0), 3.999, 13, facecolor = 'gainsboro',fill=True, alpha=0.5))
plt.plot(rs_bulk, cdf_bulk, '--', label = 'bulk', lw = 1, color='k')
plt.plot(rs_r1, cdf_r1, label = r'$0<z<4$', color = c1, lw = 1.5)
plt.plot(rs_r2, cdf_r2, label = r'4<z<5', color = c2, lw = 1.5)
plt.plot(rs_r3, cdf_r3, label = r'$5<z<6.2$', color = c3, lw = 1.5)
plt.plot(rs_r4, cdf_r4, label = r'$z>6.2$',  color = c4, lw = 1.5)
#plt.legend(loc=2)
ax2 = ax1.twinx()
ax2.plot(rs_bulk, rdf_bulk,  ':', label = 'bulk', lw = 2, color='k')
ax2.plot(rs_r1, rdf_r1, ':', label = r'$0<z<4$', color = c1, lw = 1.5)
ax2.plot(rs_r2, rdf_r2, ':', label = r'4<z<5', color = c2, lw = 1.5)
ax2.plot(rs_r3, rdf_r3, ':', label = r'$5<z<6.2$', color = c3, lw = 1.5)
ax2.plot(rs_r4, rdf_r4, ':', label = r'$z>6.2$',  color = c4, lw = 1.5)
#plt.grid()
ax1.set_xlabel(r'Cs - O distance [$\AA$]')
ax1.set_ylabel('cumm. distribution funct.')
ax2.set_ylabel('RDF')
ax1.set_ylim(0,12)
ax2.set_ylim(0,5)
ax1.set_xlim(2.75,4.5)
axins = zoomed_inset_axes(ax1, 1.4,loc=4, bbox_to_anchor=(0.05,0.56,0.5,0.5),bbox_transform=ax1.transAxes)
axins.add_patch(Rectangle((0, 0), 3.999, 13, facecolor = 'gainsboro',fill=True, alpha=0.5))
axins.plot(rs_bulk, cdf_bulk, '--', label = 'bulk', lw = 1, color='k')
axins.plot(rs_r1, cdf_r1, label = r'$0<z<4$', color = c1, lw = 1.5)
axins.plot(rs_r2, cdf_r2, label = r'4<z<5', color = c2, lw = 1.5)
axins.plot(rs_r3, cdf_r3, label = r'$5<z<6.2$', color = c3, lw = 1.5)
axins.plot(rs_r4, cdf_r4, label = r'$z>6.2$',  color = c4, lw = 1.5)
axins.add_patch(Rectangle((0, 0), 3.999, 13, facecolor = 'gainsboro',fill=True, alpha=0.5))
mark_inset(ax1, axins, loc1=3, loc2=1, fc="none", ec="0.5")
axins.grid()
axins.set_xlim(3.75, 4.25)
axins.set_ylim(6.5, 9.5)
#ax1.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
plt.tight_layout()
fig.savefig('figures/cdf_rdf.svg', format='svg', dpi=300)

Conclusion and explanation: Here, the z>6.2 region is dotted because of the bad statistics as discussed before. Already for regions 3 and 2, the CN at r_cut = 4 seems slightly reduced, but only by about 0.5. This may just be statistics. The difference becomes even larger though for ions coming even closer. As, additionally, it is the same ions that switch constantly from region 1 to 2, this difference should be considered statistically relevant!

#### same for the 2Cs case

In [ ]:
plt.plot(rs_bulk, cdf_bulk, label = 'bulk', color = 'k')
plt.plot(rs_2_r2, cdf_2_r2, label = 'region 2', color = c2)
plt.plot(rs_2_r3, cdf_2_r3, label = 'region 3', color = c3)
plt.plot(rs_2_r4, cdf_2_r4, label = 'region 4', color = c4)
plt.legend()
plt.grid()
plt.xlim(2.75,4.5)
plt.ylim(0,11)

Conclusion: Somewhat similar to what we have seen for the 4Cs case, the CN is reduced at r_cut=4A for region 2. However, similarly to above, I am a bit heasitant to put too much meaning to this, as, here, the statistics for region 2 are bad (only 3000fs - see above)

### Opening anlge

In [ ]:
r_cut = 3.999
every = 1
thetas_binned = {}
thetas, n_thetas_1 = get_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 0, zmax = 4, every = every)
thetas, n_thetas_2 = get_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 4, zmax = 5, every = every)
thetas, n_thetas_3 = get_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 5, zmax = 6.2, every = every)
thetas, n_thetas_4 = get_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 6.2, zmax = 100, every = every)

c4 = 'turquoise'
c3 = 'darkturquoise'
c2 = 'teal'
c1 = 'darkslategrey'
fig, ax1 = plt.subplots(figsize=(2.8,2))
#ax2 = ax1.twinx()
ax1.plot(thetas, 10*np.sin(thetas*np.pi/180)/np.sum(np.sin(thetas*np.pi/180))*1/(thetas[1]-thetas[0]), ':', color = 'k')
ax1.plot( np.insert(thetas, 0, 0, axis=0), 10*np.insert(n_thetas_4, 0, 0, axis=0), color = c4, label = r'$6.2<z$', lw = 1.5)
ax1.plot(thetas, 10*n_thetas_3, color = c3, label = r'$5<z<6.2$', lw = 1.5)
ax1.plot(thetas, 10*n_thetas_2, color = c2, label = r'$4<z<5$', lw = 1.5)
ax1.plot(thetas, 10*n_thetas_1, color = c1, label = r'$0<z<4$', lw = 1.5)
#ax1.fill_between(thetas, 0, n_thetas_4, color = c4, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_3, color = c3, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_2, color = c2, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_1, color = c1, alpha = 0.3)
ax1.legend()
ax1.set_xlabel(r'$\theta$ [deg]')
ax1.set_ylabel(r'$p(\theta)$ [arb. u.]');
plt.tight_layout()
fig.savefig('figures/opening_angle.svg', format='svg', dpi=300)

In [ ]:
c4 = 'turquoise'
c3 = 'darkturquoise'
c2 = 'teal'
c1 = 'darkslategrey'
fig, ax1 = plt.subplots(figsize=(10,4))
#ax2 = ax1.twinx()
ax1.plot(thetas, 10*np.sin(thetas*np.pi/180)/np.sum(np.sin(thetas*np.pi/180))*1/(thetas[1]-thetas[0]), ':', color = 'k')
ax1.plot( np.insert(thetas, 0, 0, axis=0), 10*np.insert(n_thetas_4, 0, 0, axis=0), color = c4, label = r'$6.2<z$', lw = 1.5)
ax1.plot(thetas, 10*n_thetas_3, color = c3, label = r'$5<z<6.2$', lw = 1.5)
ax1.plot(thetas, 10*n_thetas_2, color = c2, label = r'$4<z<5$', lw = 1.5)
ax1.plot(thetas, 10*n_thetas_1, color = c1, label = r'$0<z<4$', lw = 1.5)
#ax1.fill_between(thetas, 0, n_thetas_4, color = c4, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_3, color = c3, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_2, color = c2, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_1, color = c1, alpha = 0.3)
ax1.legend()
ax1.set_xlabel(r'$\theta$ [deg]')
ax1.set_ylabel(r'$p(\theta)$ [arb. u.]');

#### Same as above, but only minimum angle

In [ ]:
r_cut = 3.999
every = 1
thetas_binned = {}
thetas_min, n_thetas_1_min = get_min_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 0, zmax = 4, every = every)
thetas_min, n_thetas_2_min = get_min_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 4, zmax = 5, every = every)
thetas_min, n_thetas_3_min = get_min_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 5, zmax = 6.2, every = every)
thetas_min, n_thetas_4_min = get_min_opening_angle(atoms, 'Cs', r_cut = r_cut, nbins = 36, zmin = 6.2, zmax = 100, every = every)

c4 = 'turquoise'
c3 = 'darkturquoise'
c2 = 'teal'
c1 = 'darkslategrey'
fig, ax1 = plt.subplots(figsize=(2.8,2))
#ax2 = ax1.twinx()
ax1.plot(thetas_min, 10*np.sin(thetas_min*np.pi/180)/np.sum(np.sin(thetas_min*np.pi/180))*1/(thetas_min[1]-thetas_min[0]), ':', color = 'k')
ax1.plot( np.insert(thetas_min, 0, 0, axis=0), 10*np.insert(n_thetas_4_min, 0, 0, axis=0), color = c4, label = r'$6.2<z$', lw = 1.5)
ax1.plot(thetas_min, 10*n_thetas_3_min, color = c3, label = r'$5<z<6.2$', lw = 1.5)
ax1.plot(thetas_min, 10*n_thetas_2_min, color = c2, label = r'$4<z<5$', lw = 1.5)
ax1.plot(thetas_min, 10*n_thetas_1_min, color = c1, label = r'$0<z<4$', lw = 1.5)
#ax1.fill_between(thetas, 0, n_thetas_4, color = c4, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_3, color = c3, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_2, color = c2, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_1, color = c1, alpha = 0.3)
ax1.legend()
ax1.set_xlabel(r'$\theta$ [deg]')
ax1.set_ylabel(r'$p(\theta)$ [arb. u.]');
plt.tight_layout()
fig.savefig('figures/opening_min_angle.svg', format='svg', dpi=300)

In [ ]:
c4 = 'turquoise'
c3 = 'darkturquoise'
c2 = 'teal'
c1 = 'darkslategrey'
fig, ax1 = plt.subplots(figsize=(15,4))
#ax2 = ax1.twinx()
ax1.plot(thetas_min, 10*np.sin(thetas_min*np.pi/180)/np.sum(np.sin(thetas_min*np.pi/180))*1/(thetas_min[1]-thetas_min[0]), ':', color = 'k')
ax1.plot( np.insert(thetas_min, 0, 0, axis=0), 10*np.insert(n_thetas_4_min, 0, 0, axis=0), color = c4, label = r'$6.2<z$', lw = 1.5)
ax1.plot(thetas_min, 10*n_thetas_3_min, color = c3, label = r'$5<z<6.2$', lw = 1.5)
ax1.plot(thetas_min, 10*n_thetas_2_min, color = c2, label = r'$4<z<5$', lw = 1.5)
ax1.plot(thetas_min, 10*n_thetas_1_min, color = c1, label = r'$0<z<4$', lw = 1.5)
#ax1.fill_between(thetas, 0, n_thetas_4, color = c4, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_3, color = c3, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_2, color = c2, alpha = 0.0)
#ax1.fill_between(thetas, 0, n_thetas_1, color = c1, alpha = 0.3)
ax1.legend()
ax1.set_xlabel(r'$\theta$ [deg]')
ax1.set_ylabel(r'$p(\theta)$ [arb. u.]');
ax1.locator_params(nbins=72, axis='x')
ax1.grid()
